In [ ]:
from ray.rllib.utils.framework import try_import_tf
tf1, tf, tfv = try_import_tf()
tf1.enable_eager_execution()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ray
from ray.rllib.algorithms.algorithm import Algorithm

from esc.epanet_env import EPANETEnv


In [ ]:
algo = Algorithm.from_checkpoint('/Users/maxdumas/ray_results/PPO_EPANETEnv_2022-12-04_19-03-289hjken8u/checkpoint_000021')

In [ ]:
algo.get_policy().config["model"]

In [ ]:
env = EPANETEnv({})
obs = env.reset()
done = False
O = []
A = []
R = []

Z = np.zeros(algo.get_policy().config["model"]["lstm_cell_size"], np.float32)
state=[Z, Z] 

prev_action = 0
prev_reward = 0

while not done:
    action, state, _ = algo.compute_single_action(observation=obs, state=state, prev_action=prev_action, prev_reward=prev_reward)
    obs, reward, done, _ = env.step(action)
    O.append(obs)
    A.append(action)
    R.append(reward)
    prev_action = action
    prev_reward = reward

O = np.array(O)
A = np.array(A)
R = np.array(R)


algo.stop()

In [ ]:
plt.rcParams['figure.figsize'] = [8.0, 4.0]
plt.rcParams['figure.dpi'] = 140

In [ ]:
env.n_switches

In [ ]:
A

In [ ]:
fig, ax1 = plt.subplots()
ln1 = ax1.plot(O[:, 0] * A, label="Cost Rate ($/min)")
ax2 = ax1.twinx()
ln2 = ax2.plot(np.cumsum(O[:, 0] * A), label="Total Cost ($)", color="r")
ax2.grid()
ax1.grid()
ax1.legend(ln1 + ln2, [l.get_label() for l in ln1 + ln2], loc=2)
ax1.set_xlabel("Minute of Simulation")
ax1.set_ylabel("$/min")
ax2.set_ylabel("$")
plt.title("Pump Energy Cost")

In [ ]:
plt.plot(O[:, 2])
plt.grid()
plt.title("Tank Head (m)")

In [ ]:
# plt.plot(O[:, -1])
# plt.grid()
# plt.title("Number of Pump Actions")

In [ ]:
fig, ax1 = plt.subplots()
ln1 = ax1.plot(R, label="Instant Reward")
ax2 = ax1.twinx()
ln2 = ax2.plot(np.cumsum(R), label="Cumulative Reward", color="r")
ax2.grid()
ax1.grid()
ax1.legend(ln1 + ln2, [l.get_label() for l in ln1 + ln2], loc=2)
ax1.set_xlabel("Minute of Simulation")
plt.title("Reward")